In [1]:
%load_ext autoreload
%autoreload 2

import gradio

In [2]:
def test(x):
    return x.upper()

In [3]:
iface = gradio.Interface(input="textbox", output="textbox", model=test)

Model type not explicitly identified, inferred to be: python function


In [4]:
iface.launch()

NOTE: Gradio is in beta stage, please report all bugs to: a12d@stanford.edu
Model available locally at: http://localhost:7861/interface.html
To create a public link, set `share_link=True` in the argument to `launch()`


127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /interface.html HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /js/all-io.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /css/draw-a-digit.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /js/bootstrap-notify.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /js/textbox-input.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:01] "GET /js/textbox-output.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:06:35] "GET /interface.html HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2019 22:06:35] "GET /js/all-io.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2019 22:06:35] "GET /css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2019 22:06:35] "GET /css/draw-a-digit.css HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2019 22:06:35] "GET /js/bootstrap.min.js

In [5]:
iface2 = gradio.Interface(input="textbox", output="textbox", model=test)

Model type not explicitly identified, inferred to be: python function


In [7]:
iface = gradio.Interface(input="textbox", output="textbox", model=test)

Model type not explicitly identified, inferred to be: python function


In [4]:
#!/bin/env python
import sys, signal
import http.server
import socketserver
import threading

port = 7865

# Note ForkingTCPServer does not work on Windows as the os.fork() 
# function is not available on that OS. Instead we must use the 
# subprocess server to handle multiple requests
server = socketserver.ThreadingTCPServer(('localhost',port), http.server.SimpleHTTPRequestHandler )

#Ensures that Ctrl-C cleanly kills all spawned threads
server.daemon_threads = True  
#Quicker rebinding
server.allow_reuse_address = True  

# A custom signal handle to allow us to Ctrl-C out of the process
def signal_handler(signal, frame):
    print( 'Exiting http server (Ctrl+C pressed)')
    try:
        if(server):
            server.server_close()
    finally:
        sys.exit(0)

# Install the keyboard interrupt handler
signal.signal(signal.SIGINT, signal_handler)

# Now loop forever
def serve_forever():
    try:
        while True:
            sys.stdout.flush()
            server.serve_forever()
    except KeyboardInterrupt:
        pass

thread = threading.Thread(target=serve_forever)
thread.start()

127.0.0.1 - - [24/Feb/2019 21:50:49] code 404, message File not found
127.0.0.1 - - [24/Feb/2019 21:50:49] "GET /css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2019 21:50:49] code 404, message File not found
127.0.0.1 - - [24/Feb/2019 21:50:49] "GET /css/index.css HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2019 21:50:49] code 404, message File not found
127.0.0.1 - - [24/Feb/2019 21:50:49] "GET /js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2019 21:50:49] code 404, message File not found
127.0.0.1 - - [24/Feb/2019 21:50:49] "GET /img/draw-a-digit.png HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2019 21:50:49] code 404, message File not found
127.0.0.1 - - [24/Feb/2019 21:50:49] "GET /js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2019 21:50:49] code 404, message File not found
127.0.0.1 - - [24/Feb/2019 21:50:49] "GET /img/placeholder.png HTTP/1.1" 404 -


In [6]:
import socket

s = socket.socket() 
s.bind(('localhost', 7866))
s.close()

In [ ]:
1+1